In [5]:
from ctgan import load_demo
from ctgan import CTGAN
import pandas as pd



In [6]:
df_m = pd.read_csv('../dataset/minority.csv')
df = pd.read_csv('../dataset/majority_informative.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8200 entries, 0 to 8199
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      8200 non-null   int64  
 1   type            8200 non-null   int64  
 2   oldbalanceOrg   8200 non-null   float64
 3   newbalanceOrig  8200 non-null   float64
 4   oldbalanceDest  8200 non-null   float64
 5   isFraud         8200 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 384.5 KB


In [8]:
df.head()

,Unnamed: 0,type,oldbalanceOrg,newbalanceOrig,oldbalanceDest,isFraud
0,2441017,0,28488.83,35434.00,955.51,0
1,2301877,0,5393405.29,5405360.93,280003.74,0
2,4605340,0,10579.00,21995.77,20104.93,0
3,2767579,0,39659.00,47338.39,54399.40,0
4,1559211,0,4922014.34,4931816.15,41993.56,0


In [9]:
df = df.drop(columns=['Unnamed: 0'])

In [10]:
# Names of the columns that are discrete
discrete_columns = [
    'type'
]

ctgan = CTGAN(epochs=10)
ctgan.fit(df, discrete_columns)

# Create synthetic data


In [11]:
synthetic_data = ctgan.sample(1000)

In [13]:
synthetic_data.head()

,type,oldbalanceOrg,newbalanceOrig,oldbalanceDest,isFraud
0,0,384535.323524,3.944446e+05,5012.052717,0
1,3,22375.124236,2.218599e+05,131087.849740,0
2,2,87371.477171,2.926703e+07,42726.125287,0
3,1,-33000.393699,8.372308e+04,2582.018422,0
4,4,-6454.223290,8.536287e+04,17186.064353,0


In [ ]:
|